In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, sys

sys.path.append('c:\\Projects\\taxi_predicts\\src\\')

In [39]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-03-20 10:00:00')


[autoreload of config failed: Traceback (most recent call last):
  File "C:\Users\dpetrik\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\dpetrik\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "c:\Users\dpetrik\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Projects\taxi_predicts\src\config.py", line 5, in <module>
    from feature_store_api import FeatureGroupConfig, FeatureViewConfig
ImportError: cannot import name 'FeatureGroupConfig' from 'feature_store_api' (c:\Projects\taxi_predic

In [40]:
from inference import load_batch_of_features_from_store

#loading batch of features from store
features = load_batch_of_features_from_store(current_date)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.
Fetching data from 2024-02-21 10:00:00 to 2024-03-20 09:00:00


Finished: Reading data from Hopsworks, using Hive (55.94s) 
ts_data=               pickup_hour  rides  pickup_location_id
0      2024-03-02 08:00:00      0                 187
1      2024-02-22 23:00:00      0                 242
2      2024-02-24 17:00:00      0                 195
3      2024-03-01 22:00:00      0                 207
4      2024-02-29 10:00:00      0                 258
...                    ...    ...                 ...
184435 2024-03-20 09:00:00      0                 259
184436 2024-03-20 09:00:00      1                 243
184437 2024-03-20 09:00:00      0                 112
184438 2024-03-20 09:00:00      0                 118
184439 2024-03-20 09:00:00      2                  65

[178080 rows x 3 columns]


In [41]:
from inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.6693931619382161, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6693931619382161
[LightGBM] [Warning] bagging_fraction is set=0.2097048168865949, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2097048168865949


In [42]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,7.0,2024-03-20 10:00:00
1,2,8.0,2024-03-20 10:00:00
2,3,7.0,2024-03-20 10:00:00
3,4,9.0,2024-03-20 10:00:00
4,5,7.0,2024-03-20 10:00:00
...,...,...,...
260,261,2.0,2024-03-20 10:00:00
261,262,2.0,2024-03-20 10:00:00
262,263,21.0,2024-03-20 10:00:00
263,264,6.0,2024-03-20 10:00:00


In [43]:
import hopsworks
import config as config

In [44]:
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/549012


In [45]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [46]:

# connect to the feature group
feature_group = feature_store.get_or_create_feature_group(
    name='model_predictions_feature_group',
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_hour', 'pickup_location_id'],
    event_time='pickup_hour',
)

In [51]:
predictions.head(10)

,pickup_location_id,predicted_demand,pickup_hour
0,1,7.0,2024-03-20 10:00:00
1,2,8.0,2024-03-20 10:00:00
2,3,7.0,2024-03-20 10:00:00
3,4,9.0,2024-03-20 10:00:00
4,5,7.0,2024-03-20 10:00:00
5,6,7.0,2024-03-20 10:00:00
6,7,7.0,2024-03-20 10:00:00
7,8,7.0,2024-03-20 10:00:00
8,9,7.0,2024-03-20 10:00:00
9,10,2.0,2024-03-20 10:00:00


In [49]:
#we put the predictions into feature store so they can be later used 
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 265/265 | Elapsed Time: 00:08 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/549012/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x19704e0c430>, None)

In [ ]:
#we need to run this everytime the feature pipeline completes so we create one more action 